In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
torch.set_printoptions(linewidth=120)

In [33]:
train_set=torchvision.datasets.FashionMNIST(
root='./data/FashionMNIST'
, train=True
,download=True
,transform=transforms.Compose([
    transforms.ToTensor()
]))



In [34]:
train_loader= torch.utils.data.DataLoader(
    train_set, batch_size=100)

images,labels=next(iter(train_loader))
print(images.shape)
print(labels.shape)

torch.Size([100, 1, 28, 28])
torch.Size([100])


The shape above is important because we have to understand that the linear layer expects input in SamplesXFeatures format. So we later have to convert every batch to 100 X 28*28 

In [35]:
class Network(nn.Module):
    
    def __init__(self):
        super(Network,self).__init__()
        #self.conv1 = nn.Conv2d(in_channels = 1 , out_channels=6, kernel_size=5)   #First layer in_channel=1 since our input has only 1 channel
        #self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        ##Fully connectd layers below
        self.fc1=nn.Linear(in_features=28*28, out_features=720)
        self.fc2=nn.Linear(in_features=720, out_features=360)
        self.out=nn.Linear(in_features=360, out_features=10)   ##Since we have 10 classes to predict 
        
    def forward(self, t):
            # (1) input layer
            t = t



            # (2) hidden linear layer
            t = t.reshape(-1,28*28)   ##Converting 100 X 1 X 28 X 28 to batch_size X 28*28.  -1 here indicates figure out on your own
            t = self.fc1(t)
            t = F.relu(t)

            # (3) hidden linear layer
            t = self.fc2(t)
            t = F.relu(t)

            # (4) output layer
            t = self.out(t)
            #t = F.softmax(t, dim=1)

            return t
        

In [36]:
network=Network()

optimizer = optim.Adam(network.parameters(), lr=0.01)
grid = torchvision.utils.make_grid(images)
tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(network, images)

In [37]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
for epoch in range(10):
    total_loss=0
    total_correct=0
    for batch in train_loader:
        images,labels=batch
        preds=network(images)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss= total_loss+ loss.item()
        total_correct=total_correct+ get_num_correct(preds,labels)
    tb.add_scalar('Loss', total_loss, epoch)
    tb.add_scalar('Number Correct', total_correct, epoch)
    tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

    tb.add_histogram('fc1.bias', network.fc1.bias, epoch)
    tb.add_histogram('fc2.weight', network.fc2.weight, epoch)
    tb.add_histogram(
        'fc1.weight.grad'
        ,network.fc1.weight.grad
        ,epoch
    )
    print(
        "epoch:", epoch, 
        "total_correct:", total_correct, 
        "loss:", total_loss
    )
tb.close()

epoch: 0 total_correct: 48280 loss: 327.86426216363907
epoch: 1 total_correct: 51198 loss: 243.45736096799374
epoch: 2 total_correct: 51680 loss: 230.28387935459614
epoch: 3 total_correct: 52072 loss: 216.760657325387
epoch: 4 total_correct: 52405 loss: 208.59409792721272
